# Анализ пространственных данных. Домашнее задание №2

Мягкий дедлайн: __4 ноября 2020 г. 23:59__

Жесткий дедлайн (со штрафом в _50%_ от количества набранных вами за ДЗ баллов): __5 ноября 2020 г. 08:59__

Визуализация "чего-либо" __без__ выполненного основного задания оценивается в __0 баллов__

ФИО: Дронов Артем Геннадьевич

Группа: DS-12

In [1]:
import random
import re
import folium
from shapely.geometry import Polygon, Point, MultiPolygon, mapping
import geopandas as gpd
import json
import requests
from OSMPythonTools.nominatim import Nominatim
from OSMPythonTools.overpass import overpassQueryBuilder, Overpass
from rtree.index import Rtree
from haversine import haversine, Unit

overpass = Overpass()
nominatim = Nominatim()

## Задание №1. Горячая точка (алгоритм - 10 баллов, визуализация - 10 баллов).

Генерируйте рандомные точки на планете Земля до тех пор, пока не попадете на территорию ``Афганистана``

1. Вы можете использовать функции принадлжености точки полигону и расстояния от точки до полигона (в метрах)
2. Предложите не наивный алгоритм поиска (генерировать __напрямую__ точку из полигона границ Афганистана __запрещено__)

Визуализируйте пошагово предложенный алгоритм при помощи ``Folium``

## Решение:

Вытащим полигон Афганистана, а также его середину:

In [2]:
# Parse Afghanistan core data (point and polygon)
r = requests.get('https://nominatim.openstreetmap.org/search?country=Afghanistan&polygon_geojson=1&format=json').json()

afg_lon = float(r[0]['lon'])
afg_lat = float(r[0]['lat'])
afg_poly_coords = [[y, x] for x, y in r[0]['geojson']['coordinates'][0]]
afg_polygon = Polygon(afg_poly_coords)

Будем уменьшать диапазон угадывания в зависимости от положения точки до тех пор, пока полученная точка не будет принадлежать полигону Афганистана:

In [3]:
# Random search based on latitude and longtitude
left_lon = -180.0
right_lon = 180.0
top_lat = 90.0
bottom_lat = -90.0

random.seed(23)

point_lon = random.uniform(left_lon, right_lon)
point_lat = random.uniform(bottom_lat, top_lat)
point = Point(point_lat, point_lon)
points = [(point_lat, point_lon)]

while not afg_polygon.contains(point):
    left_lon = max(afg_lon - abs(point_lon - afg_lon), -180.0)
    right_lon = min(afg_lon + abs(point_lon - afg_lon), 180.0)
    top_lat = min(afg_lat + abs(point_lat - afg_lat), 90.0)
    bottom_lat = max(afg_lat - abs(point_lat - afg_lat), -90.0)

    point_lon = random.uniform(left_lon, right_lon)
    point_lat = random.uniform(bottom_lat, top_lat)
    point = Point(point_lat, point_lon)
    points.append((point_lat, point_lon))

Визуализируем полученное блуждание:

In [4]:
# visualization block
my_map = folium.Map(location=[afg_lat, afg_lon], zoom_start=5,
                     tiles='stamenterrain')
i = 1
for x, y in points:
    folium.Marker([x,y], popup='Step' + str(i), 
                    icon=folium.Icon(icon='arrow-down',color="darkpurple")).add_to(my_map)
    i = i + 1
folium.Polygon(afg_poly_coords, color="green", fill_color='lime').add_to(my_map)
folium.PolyLine(points, color="blue", weight=1.5, lineJoin='round').add_to(my_map)
my_map

Отобразим финальную точку для задания № 2.

In [5]:
# generated point
points[-1:]

[(33.27887887545621, 65.77417560211298)]

## Задание №2. Качество жизни (20 баллов).

Для измерения показателя качества жизни в точке, найденной в предыдущем задании, вам необходимо рассчитать следующую сумму расстояний (в метрах):

1. Расстояние от точки до 5 ближайших __*__ банкоматов, находящихся в стране с наибольшим количеством объектов жилой недвижимости
2. Расстояние от точки до 5 ближайших школ, находящихся в стране с наибольшим количеством аптек в столице
3. Расстояние от точки до 5 ближайших кинотеатров, наодящихся в стране с самым большим отношением числа железнодорожных станций к автобусным остановкам в южной части __**__

__*__ При поиске _N_ ближайших объектов обязательно использовать ``R-tree``

__**__ Южной частью страны является территория, находящаяся к югу от множества точек, равноудаленных от самой северной и самой южной точек страны

# Решение

Для начала зафиксируем точку из задания № 1 и получим список интересующих стран (имеющих id кратное десяти):

In [6]:
my_point = (33.27887887545621, 65.77417560211298)

In [7]:
data = overpass.query('relation["admin_level"="2"][boundary=administrative];out;').toJSON()
countries_name = []
for elem in data['elements']:
    country = elem['tags']['name:en']
    country_id = nominatim.query(country).areaId()
    if country_id is not None and country_id % 10 == 0:
        countries_name.append(country)
countries_name

['Brazil',
 'Estonia',
 'Australia',
 'Eswatini',
 'Bangladesh',
 'Syria',
 'Liberia',
 'Central African Republic',
 'Ethiopia',
 'Cameroon',
 'Tanzania',
 'Malawi',
 'Uzbekistan',
 'Honduras',
 'Portugal',
 'Azerbaijan',
 'Comoros',
 'São Tomé and Príncipe',
 'Antigua and Barbuda',
 'Monaco',
 'Kosovo',
 'Brunei',
 'Tokelau',
 'Norway']

# 2.1 

По ссылке указаны все теги, имеющие отношение к жилым домам. Будем использовать основные пять из них:

In [115]:
# https://wiki.openstreetmap.org/wiki/RU:Key:building#.D0.96.D0.B8.D0.BB.D1.8B.D0.B5
tags = ['apartments', 'detached','house', 'residential', 'terrace']
elem_types = ['node', 'way', 'relation']

Посчитаем количество объектов жилой недвижимости в странах:

In [118]:
accomodations_count = []
for country in countries_name:
    country_id = nominatim.query(country).areaId()
    cnt = 0
    for building in tags:
        for et in elem_types:
            query = overpassQueryBuilder(area=country_id, elementType=et, selector=f'"building"={building}', out='count');
            result = overpass.query(query, timeout=120);
            cnt += result.countElements()
    accomodations_count.append(cnt)

Лидеры по показателю:

In [117]:
sorted(list(zip(countries_name, accomodations_count)), key = lambda x: x[1], reverse=True)[:5]

[('Brazil', 754727),
 ('Australia', 584538),
 ('Norway', 297090),
 ('Tanzania', 295587),
 ('Portugal', 270278)]

Таким образом, интересующая нас страна - Бразилия. Посчитаем в ней число банкоматов:

In [8]:
country_id = nominatim.query('Brazil').areaId()
query = overpassQueryBuilder(area=country_id, elementType='node', selector='"amenity"="atm"', includeGeometry=True);
result = overpass.query(query, timeout=120);

С помощью Rtree найдем искомую сумму расстояний:

In [9]:
# build tree and find indexes of nearest points
tree = Rtree()
for i, x in enumerate(result.elements()):
    tree.insert(i, x.geometry().coordinates)
nearest = list(tree.nearest(my_point, 5))

# calcutate required distance
sum_distances = 0
for id in nearest:
    lon, lat = list(result.elements())[id].geometry().coordinates
    sum_distances += haversine(my_point, (lat, lon), unit=Unit.METERS)
sum_distances

55018952.123340696

# 2.2

Возьмем список всех столиц:

In [129]:
data = overpass.query(
    """
    relation["admin_level"="2"]["type"="boundary"]["boundary"="administrative"];
    node(r:"admin_centre");
    out meta;
    """).toJSON()

Одним из тэгов элементов является поля "is_in:country", с помощью которого можно было бы автоматически сопоставить заданные страны с городами. Однако, такой тэг не всегда имеется даже в крупных государствах (например, в Бразилии его нет), в результате чего оставалось всего 6 столиц. Учитывая, что запрос в этом пункте вычисляется довольно быстро, будем использовать все столицы.

In [157]:
capitals = []
i = 0
for elem in data['elements']:
    if 'name:en' in elem['tags']:
        cap = elem['tags']['name:en']
        capitals.append(cap)

Считаем число аптек в столице и выводим самые насыщенные:

In [170]:
pharmacy_count = []
for capital in capitals:
    try:
        id = nominatim.query(capital).areaId()
        query = overpassQueryBuilder(area=id, elementType='node', selector='"amenity"="pharmacy"', out='count');
        result = overpass.query(query, timeout=120).countElements()
        pharmacy_count.append(result)
    except:
        pass

In [166]:
sorted(list(zip(capitals, pharmacy_count)), key = lambda x: x[1], reverse=True)[:5]

[('Tallinn', 6786),
 ('Tokyo', 3184),
 ('Bratislava', 3032),
 ('Muscat', 2302),
 ('Madrid', 1797)]

Из списка можно достать столицу из данного перечня стран, однако без потери общности можно взять и Эстонию (столица Таллин).

Вычисляем количество школ в ней и требуемую сумму расстояний:

In [168]:
country_id = nominatim.query('Estonia').areaId()
query = overpassQueryBuilder(area=country_id, elementType='node', selector='"amenity"="school"', includeGeometry=True);
result = overpass.query(query, timeout=120);

In [169]:
# build tree and find indexes of nearest points
tree = Rtree()
for i, x in enumerate(result.elements()):
    tree.insert(i, x.geometry().coordinates)
nearest = list(tree.nearest(my_point, 5))

# calcutate required distance
sum_distances = 0
for id in nearest:
    lon, lat = list(result.elements())[id].geometry().coordinates
    sum_distances += haversine(my_point, (lat, lon), unit=Unit.METERS)
sum_distances

19990036.45715916

# 2.3

С помощью полигонов достаем географические середины стран из списка. На самом деле, по набору точек можно вычислить любой функционал удаленности, но для простоты возьмем деление серединой полигона.

In [197]:
middle_lats = []
for cnt in countries_name:
    r = requests.get(f'https://nominatim.openstreetmap.org/search?country={cnt}&polygon_geojson=1&format=json').json()
    middle_lats.append(float(r[0]['lat']))

Посчитаем соотношение числа ж/д станций к числу автобусных станций в городах.

Для автобусов свойство "bus=yes" размечено хорошо, поэтому можно воспользоваться исключительно им. Для ж/д узлов (в том числе в соответствии с рекомендациями overpass) будем брать две опции: public_transport=station и railway=platform (т.е. ограничимся пассажирскими).

In [229]:
coefs = []
for country, lat in zip(countries_name, middle_lats):
    id = nominatim.query(country).areaId()
    query = overpassQueryBuilder(area=id, elementType='node', selector='"public_transport"="station"', includeGeometry=True);
    result = overpass.query(query)
    train_stops = len([0 for x in list(result.elements()) if x.geometry().coordinates[0] < lat])

    query = overpassQueryBuilder(area=id, elementType='node', selector='"railway"="platform"', includeGeometry=True);
    result = overpass.query(query)
    train_stops += len([0 for x in list(result.elements()) if x.geometry().coordinates[0] < lat])

    query = overpassQueryBuilder(area=id, elementType='node', selector='"bus"="yes"', includeGeometry=True);
    result = overpass.query(query, timeout=120)
    bus_stops = len([0 for x in list(result.elements()) if x.geometry().coordinates[0] < lat])
    if bus_stops > 0:
        coefs.append(train_stops/bus_stops)
    else:
        coefs.append(0)

In [228]:
sorted(list(zip(countries_name, coefs)), key = lambda x: x[1], reverse=True)[:5]

[('Liberia', 3.3333333333333335),
 ('Antigua and Barbuda', 1.5),
 ('Honduras', 0.2876712328767123),
 ('Kosovo', 0.06465517241379311),
 ('Norway', 0.035283993115318414)]

Таким образом, наивысшим показателем обладает Либерия. Посчитаем заданную сумму для нее:

In [10]:
country_id = nominatim.query('Liberia').areaId()
query = overpassQueryBuilder(area=country_id, elementType='node', selector='"amenity"="cinema"', includeGeometry=True);
result = overpass.query(query, timeout=120);

In [231]:
# build tree and find indexes of nearest points
tree = Rtree()
for i, x in enumerate(result.elements()):
    tree.insert(i, x.geometry().coordinates)
nearest = list(tree.nearest(my_point, 5))

# calcutate required distance
sum_distances = 0
for id in nearest:
    lon, lat = list(result.elements())[id].geometry().coordinates
    sum_distances += haversine(my_point, (lat, lon), unit=Unit.METERS)
sum_distances

40762249.92039509

## Задание №3. Поездка по Нью-Йорку (маршрут - 20 баллов, визуализация - 10 баллов).

Добраться __на автомобиле__ от входа в ``Central Park`` __Нью-Йорка__ (со стороны ``5th Avenue``) до пересечения ``Water Street`` и ``Washington Street`` в Бруклине (откуда получаются лучшие фото Манхэттенского моста) довольно непросто - разумеется, из-за вечных пробок. Однако еще сложнее это сделать, проезжая мимо школ, где дети то и дело переходят дорогу в неположенном месте.

Вам необходимо построить описанный выше маршрут, избегая на своем пути школы. Визуализируйте данный маршрут (также добавив школы и недоступные для проезда участки дорог) при помощи ``Folium``

Данные о расположении школ Нью-Йорка можно найти [здесь](https://catalog.data.gov/dataset/2019-2020-school-point-locations)

# Решение:

Для нахождения маршрута будем использовать [openrootservice](https://openrouteservice.org/). Ключ сохранен в .txt, достанем его и инициализируем API.

In [28]:
with open('api_key.txt') as f:
    clnt_api_key = f.readline()

In [29]:
from openrouteservice import client
clnt = client.Client(clnt_api_key)

В качестве исходной точки возьмем автобусную остановку у Central Park на 5th Avenue.

В качестве конечной точки подойдет Dumbo Photozone, находящаяся на пересечении указанных улиц.

In [31]:
# node at 5th Avenue entrance (5th Avenue & East 63rd Street) (bus stop)

node = nominatim.query('5th Avenue & East 63rd Street').toJSON()
start_lon, start_lat = float(node[0]['lon']), float(node[0]['lat'])

# node at the cross of streets (Dumbo Photozone)

node = nominatim.query('Dumbo Photozone').toJSON()
finish_lon, finish_lat = float(node[0]['lon']), float(node[0]['lat'])

direction = ((start_lon, start_lat), (finish_lon, finish_lat)) 

Откроем данные школ из файла выше. У API есть ограничение на количество подаваемым точек в параметре avoid_polygons, в связи с чем использовать все школы не представляется возможным (да и не нужно). Поэтому сначала выделим окрестность маршрута, а затем постоим запретные полигоны для школ из рассматриваемого пространственного диапазона.

In [33]:
# get schools coordinates
schools = gpd.read_file('School_Point_Locations.csv').the_geom

# build avoiding area around it
# Notice: openrootservice returns httpError if i use all school, so i had to decrease their amount to actual area

d_lon = 2 * abs(finish_lon - start_lon)
d_lat = abs(finish_lat - start_lat)
schools_area_restrict = Polygon(
    (( start_lat + d_lat, start_lon + d_lon), ( start_lat + d_lat, finish_lon - d_lon),
    (finish_lat - d_lat, finish_lon - d_lon), (finish_lat - d_lat, start_lon + d_lon))
)

# now we can take only relevant points

poly_schools = []
for point in schools:
   y, x = re.findall(r'[-0-9.]+', point)
   poly = Point(float(x), float(y)).buffer(0.0002, resolution=8) 
   if schools_area_restrict.contains(poly):
    poly_schools.append(poly)

Теперь можно найти искомый маршрут.

In [34]:
# perform required root
params = {
    'coordinates': direction,
    'profile': 'driving-car', 
    'format': 'geojson',
    'preference' : 'fastest',
    'geometry': 'true',
    'options': {'avoid_polygons': mapping(MultiPolygon(poly_schools))},
        } 
direction_route = clnt.directions(**params)

Изобразим полученный маршрут.

In [36]:
route_map = folium.Map(location=([0.5 * (start_lat + finish_lat), 0.5 * (start_lon + finish_lon)]) ,tiles="Stamen Terrain", zoom_start=13)
folium.GeoJson(direction_route, name='Route').add_to(route_map)

folium.Marker([start_lat, start_lon], popup='Central Park (5th Avenue side)',
                icon=folium.Icon(icon='arrow-down',color="blue"), tooltip='Start').add_to(route_map)
folium.Marker([finish_lat, finish_lon], popup='Streets cross (Dumbo Photozone)',
                icon=folium.Icon(icon='arrow-down',color="blue"), tooltip='Finish').add_to(route_map)
for school_coords in poly_schools:
    folium.Polygon(list(school_coords.exterior.coords), color="red", fill_color='red').add_to(route_map)
# folium.Polygon(list(schools_area_restrict.exterior.coords), color="red", fill_color='red').add_to(route_map)   # test searching area
route_map